# Synthesize Patient Communications


<a href="https://colab.research.google.com/github/atlasfutures/memex-sample-public/blob/main/docs/tutorial/tutorials/synthesize-patient-communications/synthesize-patient-communications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
!mkdir ./data; \
    wget https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip --directory-prefix=./data/; \
    unzip ./data/synthea_sample_data_csv_apr2020.zip -d ./data/; \
    mv ./data/csv/* ./data/; \
    rm -rf ./data/csv/; \
    rm ./data/*zip

In [ ]:
!pip install -q memexdata

In [ ]:
MEMEX_INSTANCE_URL = "https://<YOUR_INSTANCE>.memexdata.com"
MEMEX_API_KEY = "<YOUR_API_KEY>"

In [ ]:
import os
from pathlib import Path

from memex import MemexSession

mx = MemexSession(MEMEX_INSTANCE_URL, api_key=MEMEX_API_KEY, verify_ssl=False)

In [ ]:
def absoluteFilePaths(directory):
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))


base_path = "./data/"

for fpath in absoluteFilePaths(base_path):
    non_empty = lambda file_path: any(
        line_count > 2 for line_count in [sum(1 for _ in open(file_path, "r"))]
    )
    if non_empty(fpath):
        print(f"Uploading {fpath}")
        mx.upload_dataset(open(fpath), Path(fpath).name)

In [ ]:
query = """
WITH patient_sample AS (
  SELECT * FROM patients p JOIN conditions c ON p.id = c.patient
  -- WHERE lower(description) LIKE '%%'
  ORDER BY RANDOM() 
  LIMIT 250
),
patient_encounters AS (
  SELECT 
    p.id as patient_id
    , p.birthdate
    , list({
        'id' : e.id
        , 'encounterclass': e.encounterclass
        , 'desc': e.description
        , 'start' : CAST(e.start as DATE)
        , 'end' : CAST(e.stop as DATE)
        , 'reason_code': e.reasoncode
        , 'reason_desc': e.reasondescription
        , 'provider_id': pr.id
        , 'provider_name': pr.name
    }) as encounter_list
  FROM patients p
  JOIN encounters e ON e.patient = p.id
  JOIN providers pr ON e.provider = pr.id
  GROUP BY p.id, p.birthdate
),
patient_conditions AS (
  SELECT 
    p.id as patient_id
    , p.birthdate
     , list({
      'id' : c.code
      , 'desc' : c.description
      , 'start' : CAST(c.start as DATE)
      , 'end' : CAST(c.stop as DATE)
  }) as condition_list
  FROM patients p
  JOIN conditions c ON c.patient = p.id
  GROUP BY p.id, p.birthdate
 ),
patient_medications AS (
  SELECT 
    p.id as patient_id
    , p.birthdate
     , list({
      'id' : m.code
      , 'desc' : m.description
      , 'start' : CAST(m.start as DATE)
      , 'end' : CAST(m.stop as DATE)
      , 'medication_reason_code': m.reasoncode
      , 'reason_desc': m.reasondescription
    }) as medication_list
  FROM patients p
  JOIN medications m ON m.patient = p.id
  GROUP BY p.id, p.birthdate
),
patients_joined AS (
  SELECT 
    p.id as id
    , p.first as first
    , p.last as last
    , p.birthdate as birthdate
    , e.encounter_list as encounter_list
    , c.condition_list as condition_list
    , m.medication_list as medication_list
  FROM
    patient_sample p
    JOIN patient_encounters e ON p.id = e.patient_id
    JOIN patient_conditions c ON p.id = c.patient_id
    JOIN patient_medications m ON p.id = m.patient_id
)
SELECT * FROM patients_joined LIMIT 100
"""

In [ ]:
mx.save_as_table("patients_joined", query, overwrite=True)

In [ ]:
from pydantic import BaseModel
from datetime import datetime
from typing import List


class Encounter(BaseModel):
    id: str
    encounterclass: str
    desc: str
    start: datetime
    end: datetime
    reason_code: float
    reason_desc: str
    provider_id: str
    provider_name: str


class Condition(BaseModel):
    id: int
    desc: str
    start: datetime
    end: datetime


class Medication(BaseModel):
    id: int
    desc: str
    start: datetime
    end: datetime
    medication_reason_code: float
    reason_desc: str


@mx.udf
def format_patient_history(
    first: str,
    last: str,
    birthdate: str,
    encounter_list: List[Encounter],
    condition_list: List[Condition],
    medication_list: List[Medication],
) -> str:
    import random

    # Convert BaseModel instances to dicts manually and exclude 'id' key
    encounter_dicts = [
        {k: v for k, v in e.items() if k != "id"} for e in encounter_list
    ]
    condition_dicts = [
        {k: v for k, v in c.items() if k != "id"} for c in condition_list
    ]
    medication_dicts = [
        {k: v for k, v in m.items() if k != "id"} for m in medication_list
    ]

    # sort the histories in reverse chronological order by start date
    encounter_dicts = sorted(encounter_dicts, key=lambda x: x["start"], reverse=True)
    condition_dicts = sorted(condition_dicts, key=lambda x: x["start"], reverse=True)
    medication_dicts = sorted(medication_dicts, key=lambda x: x["start"], reverse=True)

    # select the most recent encounter that has a reason description
    most_recent_reason = next(
        (e["reason_desc"] for e in encounter_dicts if e["reason_desc"]), None
    )

    # get the date of the most recent encounter
    most_recent_encounter_date = (
        encounter_dicts[0]["start"] if encounter_dicts else None
    )

    # get the provider name of the most recent encounter
    most_recent_provider = (
        encounter_dicts[0]["provider_name"] if encounter_dicts else None
    )

    # filter the condition and medication lists to only include items that occurred before the most recent encounter
    condition_dicts = [
        c for c in condition_dicts if c["start"] < most_recent_encounter_date
    ]
    medication_dicts = [
        m for m in medication_dicts if m["start"] < most_recent_encounter_date
    ]

    # get a list of active medications by their description
    active_medications = [m["desc"] for m in medication_dicts if m["end"] is None]

    # Format the active medications into a list. If the list is empty, return a message indicating that there are no active medications
    formatted_medications = "Active medications\n"
    formatted_medications += (
        "\n".join(active_medications) if active_medications else "No active medications"
    )

    # group encounters by reason and encounter class, with a list of dates on which those encounter types occurred
    # exclude reasons that are "None" or empty strings
    encounter_reasons = {}
    for e in encounter_dicts:
        if e["reason_desc"] and e["reason_desc"] not in encounter_reasons:
            encounter_reasons[e["reason_desc"]] = {}
        if e["reason_desc"]:
            if e["encounterclass"] not in encounter_reasons[e["reason_desc"]]:
                encounter_reasons[e["reason_desc"]][e["encounterclass"]] = []
            encounter_reasons[e["reason_desc"]][e["encounterclass"]].append(e["start"])

    # format the encounter reasons + classes and dates into a string, only include the month and year of the date (which is str format) in the format MM/DD
    formatted_encounter_reasons = "Encounter reasons\n"
    for reason, classes in encounter_reasons.items():
        formatted_encounter_reasons += f"{reason}\n"
        for cls, dates in classes.items():
            formatted_encounter_reasons += f"  {cls}: {', '.join(dates)}\n"

    # get a list of active conditions by their description
    active_conditions = [c["desc"] for c in condition_dicts if c["end"] is None]

    # Format the active conditions into a string. If the list is empty, return a message indicating that there are no active conditions
    formatted_conditions = "Active conditions\n"
    formatted_conditions += (
        "\n".join(active_conditions) if active_conditions else "No active conditions"
    )

    summary = f"""{first} {last}, born on {birthdate}, is communicating with their healthcare provider, {most_recent_provider}, about "{most_recent_reason}". 
    
## Below is a summary of their healthcare history:

### {formatted_medications}

### {formatted_encounter_reasons} 
 
### {formatted_conditions}"""

    return summary

In [ ]:
format_query = "SELECT id, format_patient_history(first, last, birthdate, encounter_list, condition_list, medication_list) as pt_hist FROM patients_joined"
mx.save_as_table("patient_histories_formatted", format_query, overwrite=True)

In [ ]:
@mx.prompt
def summarize_patient_history(patient_history: str) -> str:
    """Summarize the following patient history:
    {patient_history}
    """

In [ ]:
mx.get_models()

In [ ]:
summarize_query = """
WITH patients AS (
    SELECT *
    FROM patient_histories_formatted
    LIMIT 100
)
SELECT id , summarize_patient_history(pt_hist) as pt_hist_summary FROM patients LIMIT 10
"""

mx.save_as_table(
    "patient_histories_summarized",
    summarize_query,
    model="gpt-4-1106-preview",
    temperature=0.2,
    max_tokens=1000,
    overwrite=True,
)

In [ ]:
@mx.prompt
def create_portal_message(patient_summary: str) -> str:
    """Your instructions are to write a message to your healthcare provider as if you are the patient summarized below.
    - You are messaging them with questions about your current health issue, which may be symptoms, billing issues, prescription issues, etc.
    - Use colloquial language and be concise. Do not use sophisticated medical terms.

    Patient Summary:
    {patient_summary}
    """

In [ ]:
portal_query = """
WITH patients AS (
    SELECT id, pt_hist_summary
    FROM patient_histories_summarized
)
SELECT id, create_portal_message(pt_hist_summary) as portal_communication 
FROM patients LIMIT 10
"""

mx.save_as_table(
    "portal_communications",
    portal_query,
    model="gpt-4-1106-preview",
    temperature=0.2,
    max_tokens=1000,
    overwrite=True,
)